##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tutorial sobre canalizaciones de TFX simples con el conjunto de datos Penguin

***Un breve tutorial para ejecutar una canalización de TFX simple.***

Nota: Recomendamos ejecutar este tutorial en un bloc de notas de Colab, ¡no es necesario configurarlo! Simplemente haga clic en "Ejecutar en Google Colab".

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/penguin_simple.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/penguin_simple.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/tfx/penguin_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

En este tutorial basado en un bloc de notas, crearemos y ejecutaremos una canalización de TFX para un modelo de clasificación simple. El proceso constará de tres componentes esenciales de TFX: ExampleGen, Trainer y Pusher. La canalización incluye el flujo de trabajo de aprendizaje automático más mínimo, es decir, importación de datos, entrenamiento de un modelo y exportación del modelo entrenado.

Consulte [Explicación de las canalizaciones de TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para obtener más información sobre varios conceptos en TFX.

## Preparación

Primero tenemos que instalar el paquete de Python para TFX y descargar el conjunto de datos que usaremos para nuestro modelo.

### Actualización de pip

Para evitar actualizar Pip en un sistema cuando se ejecuta localmente, verifique que se esté ejecutando en Colab. Por supuesto, los sistemas locales se pueden actualizar por separado.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instalación de TFX


In [ ]:
!pip install -U tfx

### Desinstalación de shapely

TODO(b/263441833) Esta es una solución temporal para evitar un ImportError. En última instancia, debería solucionarse admitiendo una versión reciente de Bigquery, en lugar de desinstalar otras dependencias adicionales.

In [ ]:
!pip uninstall shapely -y

### ¿Reinició el tiempo de ejecución?

Si está usando Google Colab, la primera vez que ejecute la celda anterior, debe hacer clic en el botón "REINICIAR TIEMPO DE EJECUCIÓN" o usar el menú "Tiempo de ejecución &gt; Reiniciar tiempo de ejecución ..." para reiniciar el tiempo de ejecución. Esto se debe a la forma en que Colab carga los paquetes.

Verifique las versiones de TensorFlow y TFX.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### Configuración de variables

Hay algunas variables que se utilizan para definir una canalización. Puede personalizar estas variables como desee. De forma predeterminada, todos los resultados de la canalización se generarán en el directorio actual.

In [ ]:
import os

PIPELINE_NAME = "penguin-simple"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### Preparación de datos de ejemplo

Descargaremos el conjunto de datos de ejemplo para usarlo en nuestra canalización de TFX. El conjunto de datos que usamos es el [conjunto de datos Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html), que también se usa en otros [ejemplos de TFX](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin).

Este conjunto de datos tiene cuatro características numéricas:

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

Todas las características ya estaban normalizadas en un rango [0,1]. Compilaremos un modelo de clasificación que prediga las `species` de pingüinos.

Debido a que TFX ExampleGen lee entradas de un directorio, tenemos que crear un directorio y copiar el conjunto de datos en él.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

Veamos rápidamente al archivo CSV.

In [ ]:
!head {_data_filepath}

Debería poder ver cinco valores. `species` es una de 0, 1 o 2, y todas las demás características deben tener valores entre 0 y 1.

## Cómo crear una canalización

Las canalizaciones de TFX se definen mediante las API de Python. Definiremos una canalización que conste de los siguientes tres componentes.

- CsvExampleGen: lee archivos de datos y los convierte al formato interno de TFX para su posterior procesamiento. Hay varios [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen) para varios formatos. En este tutorial, usaremos CsvExampleGen, que toma la entrada del archivo CSV.
- Trainer: entrena un modelo de aprendizaje automático. Los usuarios deben aportar un código de definición de modelo al [componente Trainer](https://www.tensorflow.org/tfx/guide/trainer). Puede utilizar las API de TensorFlow para especificar cómo entrenar un modelo y guardarlo en formato *save_model*.
- Pusher: copia el modelo entrenado fuera de la canalización de TFX. El [componente Pusher](https://www.tensorflow.org/tfx/guide/pusher) se puede considerar como un proceso de implementación del modelo de ML entrenado.

Antes de definir realmente la canalización, debemos escribir un código modelo para el componente Trainer.

### Cómo escribir un código de entrenamiento modelo

Crearemos un modelo DNN simple para clasificación con ayuda de la API TensorFlow Keras. Este código de entrenamiento del modelo se guardará en un archivo separado.

En este tutorial usaremos [Trainer genérico](https://www.tensorflow.org/tfx/guide/trainer#generic_trainer) de TFX que admite modelos basados ​​en Keras. Debe escribir un archivo Python que contenga la función `run_fn`, que es el punto de entrada para el componente `Trainer`.

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Ahora ha completado todos los pasos de preparación para compilar una canalización de TFX.

### Cómo escribir una definición de canalización

Definimos una función para crear una canalización de TFX. Un objeto `Pipeline` representa una canalización de TFX que se puede ejecutar con uno de los sistemas de orquestación de canalizaciones que admite TFX.


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

## Cómo ejecutar la canalización

TFX admite múltiples orquestadores para ejecutar canalizaciones. En este tutorial usaremos `LocalDagRunner`, que se incluye en el paquete TFX para Python y ejecuta canalizaciones en el entorno local. A menudo hablamos de las canalizaciones de TFX como "DAG", que significa grafo acíclico dirigido.

`LocalDagRunner` proporciona iteraciones rápidas para desarrollo y depuración. TFX también admite otros orquestadores, incluidos Kubeflow Pipelines y Apache Airflow, que son adecuados para casos de uso de producción.

Consulte [TFX en Cloud AI Platform Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines) o el [Tutorial de TFX Airflow](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop) para obtener más información sobre otros sistemas de orquestación.

Ahora creamos un `LocalDagRunner` y pasamos un objeto `Pipeline` creado a partir de la función que ya definimos.

La canalización se ejecuta directamente y puede ver registros del progreso de la canalización, incluido el entrenamiento del modelo de aprendizaje automático.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

Debería encontrar "INFO:absl:Component Pusher is finished" al final de los registros si la canalización se completó correctamente. Porque el componente `Pusher` es el último componente de la canalización.

El componente Pusher envía el modelo entrenado a `SERVING_MODEL_DIR`, que es el directorio `serving_model/penguin-simple` si no cambió las variables en los pasos anteriores. Puede ver el resultado desde el explorador de archivos en el panel del lado izquierdo de Colab, o si usa el siguiente comando:

In [ ]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

## Siguientes pasos

Puede encontrar más recursos en https://www.tensorflow.org/tfx/tutorials.

Consulte [Explicación de las canalizaciones de TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para obtener más información sobre varios conceptos en TFX.
